In [2]:
from tensorflow import keras
from tensorflow.keras import applications


conv_base = keras.applications.vgg16.VGG16(
    weights = "imagenet",   ## Peso de control con el cual se inicializará el modelo
    include_top = False,    ## Se refiere a incluir o no un clasificador densamente conectado sobre la red, por defecto corresponde a un 
                            ## clasificador con 1000 imagenes
    input_shape = (180, 180, 3) ## Forma del tensor de la imagen
)


58889256/58889256 [==============================] - 19s 0us/step


1. Se entrena el modelo conv_base con nuestro Dataset, registra su salida en una matriz NumPy
en el disco y luego usar estos datos como entrada para un sistema independiente

2. extender el modelo conv_base y añadir una capadensa encima, esto permite hacer una aumentación de datos, pero este metodo es más
costose que el anterior


Se leen las carpetas de las imagenes creadas anteriormente

In [7]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(new_base_dir / "train",
                                              image_size = (180, 180),
                                              batch_size = 32)

validation_dataset = image_dataset_from_directory(new_base_dir / "validation",
                                              image_size = (180, 180),
                                              batch_size = 32)

test_dataset = image_dataset_from_directory(new_base_dir / "test",
                                              image_size = (180, 180),
                                              batch_size = 32)

## los datasets son del tamaño (32, 180, 180, 3)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [8]:
import numpy as np
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

1/1 [==============================] - 2s 2s/step


In [1]:
inputs = keras.Input(shape=(5,5,512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="feature_extraction.keras",
 save_best_only=True,
 monitor="val_loss")
]
history = model.fit(
 train_features, train_labels,
 epochs=20,
 validation_data=(val_features, val_labels),
 callbacks=callbacks)

NameError: name 'keras' is not defined

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")

plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

2. Extencion del modelo y aumentacion de datos

In [ ]:
## Se congela el modelo (preentrenado) para que no sea entrenado nuevamente
conv_base = keras.applications.vgg16.VGG6(weights = "imagenet",
                                          include_top = False)
conv_base.trainable = False

## Aumentación
data_augmentation = keras.Sequential(
 [
 layers.RandomFlip("horizontal"),
 layers.RandomRotation(0.1),
 layers.RandomZoom(0.2),
 ]
)
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs) 
x = keras.applications.vgg16.preprocess_input(x) 
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])

Fine-Tune  Ajuste fino

consiste en descongelar unas de las capas superiores del modelo congelado y luego se une al modelo(completo)
Se empiza desde las capas superiores para no propagar errores que se puedan generar

STEPS
1. añadir la parte descongelada a la red base entrenada
2. Congelar la red base
3. entrenar la parte añadida
4. Descongelasr algunas capas de la red base
5. Agrgar la red personalizada a una red preentrenada

Se usan las capas superiores (2 o 3) por que estas se encargan de las caracteristicas más especificas mientras que las capas inferiores 
tienen caracteristicas más genéricas

Si se entrenan las demás capar hay más probabilidades de overffiting

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
  layer.trainable = False

model.compile(loss="binary_crossentropy",
 optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
 metrics=["accuracy"])
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="fine_tuning.keras",
 save_best_only=True,
 monitor="val_loss")
]
history = model.fit(
 train_dataset,
 epochs=30,
 validation_data=validation_dataset,
 callbacks=callbacks)


model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset) 
print(f"Test accuracy: {test_acc:.3f}")